<a href="https://colab.research.google.com/github/Andres8bit/Machine-Learning/blob/main/Classification_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD,RMSprop
from keras.losses import SparseCategoricalCrossentropy
from keras.utils import to_categorical
from keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras import layers
import  numpy as np
import csv
import random
import matplotlib.pyplot as plt
from google.colab import drive
from skimage import data, io, filters
from PIL import Image
drive.mount('/content/drive')
%matplotlib inline

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
training_path = '/content/drive/My Drive/facial_expression_train_data.csv'
target_path = '/content/drive/My Drive/facial_expression_train_target.csv'
test_path = '/content/drive/My Drive/facial_expression_test_data.csv'
label = {'Angry':0,'Happy':1,'Disgust':2}


def load_set(path):
  with open(path,'r') as f:
    reader = csv.reader(f,delimiter=',')
    data = np.array(list(reader)).astype(float)
  return data

# input:
#       data =  data to split into two randomly sampled sets
#       r = ratio with which to divide the two set,
#           r is a decimal value representing percentage to be saved for validation
def split_set(data,label,r):
  size = r * data.shape[0] # same as .1 * 100 = 10
  # create a random numpy array of shape(vsize,data.row)
  #create a list of randomly sampled list of indices of the original set
  rows = random.sample(range(data.shape[0]),int(size),)
  data_set = np.zeros(shape=(int(size),data.shape[1]))
  label_set = np.zeros(shape=(int(size),1))
  for i in range(len(rows)):
    data_set[i] = data[rows[i]] #= np.append(v_set,data[rows[i]])
    label_set[i] = label[rows[i]]
  data_set = np.reshape(data_set,(int(size),48,48,1))
  label_set = to_categorical(label_set,num_classes=3)
  return data_set,label_set

X = load_set(training_path)
Y = load_set(target_path)
testing_values = load_set(test_path)
predict_values = np.reshape(testing_values,(testing_values.shape[0],48,48,1))
training_set_X,training_set_Y = split_set(X,Y,.7)
validation_X,validation_Y = split_set(X,Y,.1)
test_set_X,test_set_Y = split_set(X,Y,.2) 



In [ ]:
data_augmentation = Sequential(
    [layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(48, 48, 1)),
     layers.experimental.preprocessing.RandomRotation(0.2),
     layers.experimental.preprocessing.RandomZoom(0.2),
     layers.experimental.preprocessing.RandomContrast(0.19)]
)
#model construction:
model = Sequential()
model.add(data_augmentation)
model.add(Conv2D(16, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(16, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32,(3,3), padding='same',activation='relu'))
model.add(Conv2D(32,(3,3), padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3), padding='same', activation='relu'))
model.add(Conv2D(64,(3,3), padding='same', activation='relu'))
model.add(Conv2D(64,(3,3), padding='same', activation='relu'))
model.add(Conv2D(64,(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,(3,3), padding='same', activation='relu'))
model.add(Conv2D(128,(3,3), padding='same', activation='relu'))
model.add(Conv2D(128,(3,3), padding='same', activation='relu'))
model.add(Conv2D(128,(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(1,1), padding='same', activation='relu'))
model.add(Conv2D(64,(1,1), padding='same', activation='relu'))
model.add(Conv2D(64,(1,1), padding='same', activation='relu'))
model.add(Conv2D(64,(1,1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(3,activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
nb_epoch = 512
batch_size = 256

#lastly need to split data sets into training_set, testing_set and validation_set
model_Fit = model.fit(training_set_X, training_set_Y, epochs=nb_epoch, batch_size=batch_size,
          validation_data=(validation_X, validation_Y), shuffle=True, verbose=1)

Epoch 1/512
45/45 [==============================] - 2s 44ms/step - loss: 1.0765 - accuracy: 0.4340 - val_loss: 1.0717 - val_accuracy: 0.4360
Epoch 2/512
45/45 [==============================] - 2s 38ms/step - loss: 1.0697 - accuracy: 0.4441 - val_loss: 1.0897 - val_accuracy: 0.4360
Epoch 3/512
45/45 [==============================] - 2s 37ms/step - loss: 1.0696 - accuracy: 0.4441 - val_loss: 1.0805 - val_accuracy: 0.4360
Epoch 4/512
45/45 [==============================] - 2s 37ms/step - loss: 1.0694 - accuracy: 0.4441 - val_loss: 1.0672 - val_accuracy: 0.4360
Epoch 5/512
45/45 [==============================] - 2s 38ms/step - loss: 1.0680 - accuracy: 0.4442 - val_loss: 1.0619 - val_accuracy: 0.4360
Epoch 6/512
45/45 [==============================] - 2s 38ms/step - loss: 1.0672 - accuracy: 0.4439 - val_loss: 1.0608 - val_accuracy: 0.4360
Epoch 7/512
45/45 [==============================] - 2s 38ms/step - loss: 1.0631 - accuracy: 0.4438 - val_loss: 1.0669 - val_accuracy: 0.4416
Epoch 

In [ ]:
results = model.evaluate(test_set_X,test_set_Y)
print("test lost, test acc:",results)

102/102 [==============================] - 0s 4ms/step - loss: 0.3951 - accuracy: 0.8461
test lost, test acc: [0.39513519406318665, 0.84605872631073]


In [ ]:
predict = model.predict(predict_values)
values = np.argmax(predict,axis=-1)
index = np.arange(0,predict.shape[0],1)#create a 1D array of index values by only taking the most likey value.
submission = np.expand_dims(index,axis=1)
submission = np.stack((index,values),axis=1)
np.savetxt("submission_20.csv",submission,delimiter=',',header="Id,Category",fmt="%d,%d")